In [70]:
import numpy as np

In [71]:
def relu(Z):
  return np.maximum(0, Z)


def relu_backward(dA, Z):
  return np.where(Z > 0, dA, 0)


relu(np.array([[-5, 4], [0, 1]]))  # relu(Z)

relu_backward(
    np.array([[-0.2, -0.5], [-0.1, 0.2]]),  # dA
    np.array([[-5, 4], [0, 1]])  # Z
)

array([[0, 4],
       [0, 1]])

array([[ 0. , -0.5],
       [ 0. ,  0.2]])

In [72]:
def sigmoid(Z):
  return 1 / (1 + np.exp(-Z))


def sigmoid_backward(dA, Z):
  s = sigmoid(Z)
  return dA * s * (1 - s)


sigmoid(np.array([[-5, 4], [0, 1]]))  # sigmoid(Z)

sigmoid_backward(
    np.array([[-0.2, -0.5], [-0.1, 0.2]]),  # dA
    np.array([[-5, 4], [0, 1]])  # Z
)

array([[0.00669285, 0.98201379],
       [0.5       , 0.73105858]])

array([[-0.00132961, -0.00883135],
       [-0.025     ,  0.03932239]])

In [73]:
def bce_loss(A, Y):
  A = np.clip(A, 1e-9, 1 - 1e-9)
  return -np.mean(Y*np.log(A) + (1-Y)*np.log(1-A))


def bce_loss_backward(A, Y):
  n = Y.shape[0]
  A = np.clip(A, 1e-9, 1 - 1e-9)
  return (-Y/A + (1-Y)/(1-A)) / n


bce_loss(
    np.array([0.2, 0.1, 0.8, 0.9]),  # A
    np.array([1, 0, 1, 1])  # Y
)

bce_loss_backward(
    np.array([0.2, 0.1, 0.8, 0.9]),  # A
    np.array([1, 0, 1, 1])  # Y
)

np.float64(0.5108256237659906)

array([-1.25      ,  0.27777778, -0.3125    , -0.27777778])

In [96]:
class LinearLayer:
  def __init__(self, n_inputs, n_neurons):
    self.W = np.random.randn(n_inputs, n_neurons) * 0.01
    self.b = np.zeros(n_neurons)

    self.X = None
    self.dW = None
    self.db = None

  def forward(self, X):
    self.X = X
    return X @ self.W + self.b

  def backward(self, dOut):
    self.dW = self.X.T @ dOut